# 第6章　様々な音響信号処理（確認問題の解答）
コードのセルを，本文ノートブック chap6.ipynb にコピペして実行してください。


## 6.1 ボイスチェンジャ 

(1) ボイスチェンジのためにかけた正弦波の周波数が f = 10 [Hz] の場合，元の音声と異なる聞こえにはなるが，「その人の声」という意味では変わらない感じがしたであろう。このことを，スペクトルを観察した結果と関係して考察しなさい。

【答】ここで求めているのは 512 点（32 ms）区間のスペクトルです。f = 10 [Hz] の場合には，100 ms の周期で音声の振幅が大→小→（負の）大→小→大とゆっくり変化するので，32 ms の区間内では振幅の変化がさほど大きくありません。つまり，元の音声の振幅全体が「ちょっと小さめ」や「正負反転」のような波形が観測されるので，その区間のスペクトルの外形は，元の音声から大きく変化しません。それゆえ，「その人の声」という性質は保持されます。

一方で，f = 10 [Hz] というゆっくりした変化では，音の大小の変化を感じ取ることができます。この場合に感じた元の音声からの音色の変化は，この時間構造の変化によるところの効果が大きいといえます。

(2) 上記の (c) では，（これは「平衡変調」の話に関連させたかったので）キャリアとして「単一周波数の正弦波」を用意した。実際の音声について考えるために，そのキャリアを f0 = 1000 [Hz] に加えて 2000 Hz 成分ももつ２成分音として，スペクトルを観察しなさい。

【答】(c) の最初のセルで，キャリアを合成する行を以下のとおり編集するだけです。ぜひ，f = 200 [Hz] 以外の値でも試してください。

In [ ]:
wave_data = np.sin(2 * np.pi * f0 * t) + np.sin(2 * np.pi * 2 * f0 * t) 

(3) ここで試した方法以外のボイスチェンジャの作戦を調べること。

【答】種々の方法があります。例えば，以下です。

・時変平衡変調：本文では 例えば f = 200 [Hz] を固定した平衡変調をかけましたが，f = 200 + 100 * sin(2 \pi t) のように f を時間と共に変化させます。

・ヘリウム音声の模擬：7章でボコーダによる音声合成を行います。線形予測係数を求める前に，スペクトルを周波軸上で 3 倍に引き延ばします（線形補間で十分でしょう）。その後に，ボコーダによる音声合成を行えば，ヘリウムガスを吸引・発声したときの声に近くなります。

・スペクトルピーク音声：FFTスペクトルで極大値の周辺のみを残して逆FFTすると，フォルマント情報は残りますので，音韻は変わらず音色が変わります。

・リング変調：リングバッファを利用し，話速は変化させずピッチだけを高く／低くします。

ところで，ボイスチェンジャ（話速が変わらず音色が変わる）の反対で，話速変換（音色は変わらず話速が変わる）という技術もあります。NHK が語学学習用スマホアプリとして配布しており，どのような技術か公開していますので興味のある方は検索してみてください。


## 6.2 エフェクタ

(1) ここで試した以外に，エフェクタ／エフェクトとして知られているものを調べること。

【答】種々のエフェクタ／エフェクトがあります。例えば，以下です。

・ディストーション：歪を発生させます。最も簡単なハードクリッピングでは，波形の絶対値がある値を超えたら，頭打ちにしてしまいます。それゆえ，リミッタとも呼ばれます。

・コンプレッサ：上記のリミッタでは特定の値を上回ると定数にしてしまいますが，コンプレッサーでは特定の値を上回った波形を一定の振幅比で圧縮します。小さい音が相対的に大きくなるので，存在感が上がると評価する人もいます。

・ワウ：BPFの通過帯域を時間とともに変化させてフィルタリングすることで，「ワ」「ウ」と聞こえるような効果があります。

・コーラス：ビブラートなどにより音色に色づけした音を，原音に重ねます。これにより，独唱データから疑似的な合唱データを生成できます。


(2) ディストーションと呼ばれるエフェクトを実装して，聴感上の効果を確認すること。

【答】インプット 6.6 を修正してみます。

In [ ]:
# インプット 6.6 を修正したもの
fs, wave_data = scipy.io.wavfile.read ('sample/sample3.wav')  # 音楽データを読み込みます。

sampling_interval = 1.0 / fs                  # サンプリング間隔は，サンプリング周波数の逆数
times = np.arange ( len ( wave_data )) * sampling_interval  # サンプル系列の時刻データの配列

plt.subplot(211)
plt.title("Output 7.6")
plot_wave(times, wave_data) 

# 以下で，ディストーションをかけます。
limit = 2000.0   # この値を様々に変えて効果を確認してください。
distortion = np.copy(wave_data)
distortion[ distortion >=  limit] =  limit 
distortion[ distortion <= -limit] = -limit 

plt.subplot(212)
plot_wave(times, distortion) 

audio = Audio(distortion, rate = fs)
audio

## 6.3 マイクロホンアレイによるビームフォーミング

(1) ここでは，雑音の低減を耳で確認してきた。一般には，雑音低減量 [dB] は以下で定義される。
$$
雑音低減量 {\rm [dB]} = 10 \log_{10} \frac{処理後の雑音パワー}{処理前の雑音パワー}
$$
「処理前」と「処理後」2つの音を与えたときに低減量を算出する関数を定義して，実際に「(a) ステップ3」について雑音低減量を計算しなさい。

【答】以下に「処理前」と「処理後」の2つの音を与えた場合に低減量を算出する関数 supression_cal の定義を示します。

In [ ]:
# このセルを下記の手順に従い本文にコピーして，計算してみてください。
def suppression_cal(before, after):
    '''2音の間での低減量を算出する関数
       引数 before：基準となる音データ
            after:  処理の効果を評価したい音データ
    '''
    before_power = np.sum(before**2) / len(before)
    after_power = np.sum(after**2)   / len(after)
    return 10 * np.log10(after_power / before_power)

具体的手順は以下です。

1. 本文ノートブックに「+コード」アイコンで新しいセルを作り，上記の関数定義をそのセルにコピーして，実行することで関数定義を行います。

1. 本文ノートブックにおける「# インプット 7.12 （ステップ (1) のコード）」のセルを再度実行します。

1. 本文ノートブックにおける「# ステップ (2) のコード」というセルにおいて，以下の2行を捜し，wave_data に 0.0 をかけます（着目したいのは雑音だけですので，信号は不要です）。そのうえで，そのセルを実行します。
```
for channel_id in range(n_channels):
    z[channel_id] = circular_delay(noise, fs, channel_id * tau) + 0.0 * wave_data # 遅延した雑音と，目的音を重ねます。
```

1. 本文ノートブックにおける「# ステップ (3-a) 録音された全体を一気に処理する場合」というセルにおいて，y を計算した後に以下の2行を追加します。そのうえで，そのセルを実行します。
```
suppression = suppression_cal(noise, y)
print("Amound of noise suppression =", suppression, "[dB]")
```
1. 出力された値は，約 -11 dB という値になりましたか？確認してください。

(2) すでに，雑音の到来方向 (DOA) を変えた場合に，雑音低減量が異なることは音を聞くことで観測済みである。雑音の DOA を$-90^\circ$ から$90^\circ$まで　$1^\circ$ ごとに変え，雑音低減量を計算しなさい。横軸を DOA とし，縦軸を雑音低減量としたグラフ（空間スペクトルやビームパターンと呼ばれる）を作成すること。

【答】「# ステップ(2)のコード」というセルを以下のように加筆・修正したものを以下に示します。 本文ノートブックにおける適当な場所に，「+コード」アイコンで新しいセルを作り，そこに以下のコードをコピーして実行してください。

In [ ]:
# ステップ (2) のコード  → 確認問題用に加筆修正
n_channels = 20     # マイクロホンの個数
d = 7e-3            # マイクロホンの間隔 [m]
c = 340.0           # 音速 [m/s]
noise_amp = 1000.0  # 雑音の振幅

n_samples = 3000                                # 実行時間短縮のため，ポイント数は3000に減らす。
z = np.zeros((n_channels, n_samples))           # まず「全マイクロホンの観測データ」を納める，2次元配列を準備します。
noise = noise_amp * np.random.randn(n_samples)  # 雑音データを準備します。

# --- (この部分は新設) ---
suppression = np.zeros(181)  # -90度から90度まで1度刻みなので 181 角度について記録する。

# --- マイクロホン間の遅延を計算して，他のマイクロホンに到達した雑音を計算します。 ----
for DOAindex in range(-90, 91, 1):
    DOA = DOAindex / 180.0 * np.pi 
    tau = d * np.sin(DOA) / c                      # マイクロホン間の遅延時間を計算します。
    for channel_id in range(n_channels):
        z[channel_id] = circular_delay(noise[:n_samples], fs, channel_id * tau) #+ wave_data # 遅延した雑音と，目的音を重ねます。

    # --- 以下はステップ (3-a) からのコピー ---
    y = np.zeros(n_samples)                # アレイからの出力を納める配列を用意して，
    for channel_id in range(n_channels):   # 全てのチャネルについて
        y += z[channel_id]                 # 単に加算します。 
    y = y / n_channels                     # 最後にチャネル総数で除せば，処理は完了です。
    suppression[DOAindex+90] =  suppression_cal(noise, y) # 結果を記録

# --- (この部分は新設) ---
plt.plot(np.arange(-90, 91,1), suppression)
plt.xlabel('DOA of noise (deg.)')
plt.ylabel('Noise suppression (dB)')
plt.xticks(np.arange(-90,91,30))
plt.show()

(3) マイクロホンの間隔を広めに設定して，空間折返し歪が生じる条件では，上記のグラフはどのような特徴をもつか観測すること。ただし，到来信号としては正弦波を用いること（その周波数を様々に変化させて，検討すること）。

【答】上記のセルで d = 7e-3 (7 mm) となっている数値を，例えば d = 0.10 (10 cm)と変えます。また，
```
noise = noise_amp * np.random.randn(n_samples)
```
の1行を
```
f = 1000 # [Hz]
t = np.arange(n_samples) * sampling_interval
noise = noise_amp * np.sin(2 * np.pi * f * t)
```
の3行に変更します。まだこの設定であれば，空間折返し歪は発生しません（1000 Hz の波長は 34 cm なので，その半波長である 17 cm 以下であれば発生しません）。でも，f = 5000 [Hz] とすると，波長は 6.8 cm であり，半波長 3.4 cm は 10 cm より短いので，しっかりと折返し歪が発生します（DOA が 0$^\circ$以外にも消去されない場合が発生します）。

(4) ここで取り上げた遅延和アレイでは，$M$個のマイクロホンについて，各マイクロホンからの出力に $1/M$ という重みをかけて加算したと見なすことができる（重みの和は 1 である）。これは，「マイクロホン出力の系列を【方形波窓】で切り出した」のと等価である。では，例えば「ハニング窓をかけて切り出す（すなわち，両端のマイクロホンほど重みを小さくする）」をした場合に，(2) で観測したグラフはどのように変化するか，観測すること。

【答】上記のセルにおける

``` 
    # --- 以下はステップ (3-a) からのコピー ---
    y = np.zeros(n_samples)                # アレイからの出力を納める配列を用意して，
    for channel_id in range(n_channels):   # 全てのチャネルについて
        y +=  z[channel_id]                # 単に加算します。 
```

の部分を以下のように加筆します。すなわち，窓関数 w をハニング窓により準備して，かけてから加算するようにします。

``` 
    # --- 以下はステップ (3-a) からのコピー ---
    y = np.zeros(n_samples)                # アレイからの出力を納める配列を用意して，
    w = hanning( n_channels )              # 窓関数を用意して，
    for channel_id in range(n_channels):   # 全てのチャネルについて
        y += w[channel_id] * z[channel_id] # 窓掛けをしながら加算します。 
```

このように窓をかけることで「スペクトルのメインローブが広くなり，サイドローブが低くなる」という変化が見られたでしょう。f を高めに設定（例えば f = 3000）すると効果がよく分かります。

(5) マイクロホンアレイの性能を評価するための代表的な尺度は，上記の雑音低減量である。もう一つの代表的な尺度として，SDR (Signal-to-Distortion Ratio) がある。
$$
{\rm SDR \ [dB]} = 20 \log_{10} \frac{\sum_{f} \left|X(f) \right|}{\sum_{f} \left| \left| X(f) \right| - \left| Y(f) \right| \right|
}
$$
ここで，$X(f)$, $Y(f)$ は，それぞれ「本来の目的音のスペクトル」と「処理後に得られた目的音のスペクトル」である。定義から明らかなように，もしpワースペ苦トルとして全く歪がない（$|X(f)| =|Y(f)|$）とすれば分母は0となるので，SDR は $\infty$ dB となる。この尺度を算出する関数を定義し，実際に (a) の遅延和アレイを実行する前後でSDR はどの程度改善したか，算出してみること。

【答】以下に「処理前」と「処理後」の2つの音を与えた場合に低減量を算出する関数 SDR_cal の定義を示します。

In [ ]:
def SDR_cal(x, y):
    ''' SDR を計算する関数の定義
        引数 x: 原音のデータ
             y: 処理後の音のデータ
    '''
    nsamples = min(len(x), len(y))            # x と y のうち短い系列の点数を計算対象とします。
    Xabs = np.abs(np.fft.fft(x[ : nsamples])) # x のスペクトルを求めて，絶対値を計算します。
    Yabs = np.abs(np.fft.fft(y[ : nsamples])) # y のスペクトルを求めて，絶対値を計算します。

    SDR = 20 * np.log10( np.sum(Xabs) / np.sum(np.abs(Xabs - Yabs))) # 定義どおりに SDR を計算します。
    return SDR

具体的手順は以下です。

1. 本文ノートブックに「+コード」アイコンで新しいセルを作り，上記の関数定義をそのセルにコピーして，実行することで関数定義を行います。

1. 本文ノートブックにおける「# ステップ (1) のコード」，「# ステップ (2) のコード」，「# ステップ (3-a) 録音された全体を一気に処理する場合」を念のため実行します。もし課題(1)で，「# ステップ (2) のコード」というセルにおいて，wave_data に 0.0 をかけるように修正したら，その「0.0 *」をトルしてください。

1. 本文ノートブックに「+コード」アイコンで新しいセルを作り，
```
SDR_cal(wave_data, z[0])
```
を実行することで，処理前の SDR を計算します。

1. 本文ノートブックに「+コード」アイコンで新しいセルを作り，
```
SDR_cal(wave_data, y)
```
を実行することで，処理前の SDR を計算します。

1. SDRは，処理前：約 $-10$ dB，処理後：約$9$ dB となりましたか？確認してください。

1. さらに「# ステップ (2) のコード」における「noise_amp = 1000.0  # 雑音の振幅」という値を例えば 100.0 に変えてから再度実行して，SDRの値と音質劣化の関係を確かめてください。→ SDR はどの程度の値であれば，「歪の影響はない」といえそうですか？

補足：上記の SDR の計算では， Xabs と Yabs が全周波数帯を見れば等しいことを仮定しています。もし，その仮定が成り立たない場合には，事前に Yabs のパワーが Xabsと等しくなるように調整します。

(6) 本節の冒頭で，遅延和アレイでは「マイクロホンからの出力に適切な遅延を付加して加算する」ことを述べた。その遅延量は「標本化周期より小さい値」であってもよい（**分数遅延**と呼ばれる）。それを時間領域だけで実現するとする手続きは，本節でも用いた「位相回転を利用した遅延」を利用することである。さて，分数遅延を含む遅延和アレイ処理を，周波数領域まで利用して実現する手続きを述べなさい。

【答】以下の手順で実現できます。

1. 周波数領域で設計した分数遅延フィルタを逆フーリエ変換して時間領域信号に戻す。（第5章記載の手法）

1. これに後ろに0を詰めてFFTして周波数領域で畳み込み演算を実施する。（第6章記載の方法）

1. 最後にそのフレームの時間信号の和を取る。

若干複雑な感じがしますが，周波数領域で求めたフィルタをそのまま周波数領域で掛け算するのは，円状畳み込みによる折返歪（エリアシング）を生むこと，またHanning窓を掛けることは，周波数領域では周波数方向の平均になり応答がなまる非線形処理である，といったことを回避しているのです。